In [3]:
# Basic Tools
import pandas as pd
import numpy as np



from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, cross_validate



In [4]:
data = pd.read_csv('../data/processed/data_processed.csv')